In [1]:
# google drive マウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/'My Drive'/Sprint25/dive_groupwork

/content/drive/My Drive/Sprint25/dive_groupwork


In [3]:
!git pull

Already up to date.


In [4]:
!pip install pygame

     |████████████████████████████████| 11.4MB 6.5MB/s 


In [7]:
!pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-cp36-none-any.whl size=26819 sha256=4755a6ade4c5ce46a2c8b1a6c4623e98bfc6f0971d737f415aa107bd86ff2257
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [85]:
import pygame.mixer
import numpy as np
# ラズパイ時に復活
# import picamera
from PIL import Image
from time import sleep, perf_counter

from translate import *
from bottle_master import bottle_master
# ラズパイ時に復活
# from shutter2image import shutter2image
import os
import datetime
import pandas as pd

# modelを初期値をNoneに
# time_debugモードを追加

class main:
    """
    マスタークラス

    """
    def __init__(self, model='Random', lang="j", time_debug=True):
    # def __init__(self, model="YOLO", lang="e"):
        self.model_dict = {'y':"YOLO", 'e':"EFF", 'r':"Random"}
        self.language_dict = {'k':'Kinyarwanda', 'e':'English', 'f':'French', 'j':'Japanese'}
        self.model = model
        self.lang = lang
        self.check_list = ["y", "n"]
        self.time_debug = time_debug

    def mode_select(self):
      """
      初期起動モードを選択させる関数
      標準入力から引数を取って分岐する
      """
      mode_list = ["1", "2", "3"]
      print("\n----------Current status---------")
      print("model : {}\nlanguage : {}".format(self.model, self.language_dict[self.lang]))
      
      print("\n----------Please choice start up mode----------")
      print(" 1 : Start in cash register mode")
      print(" 2 : Start in set up mode")
      print(" 3 : Start in image mode")
      print("--------------------")
      mode = input("Please select and enter the number : ")

      while mode not in mode_list:
          print("\nInvalid number.")
          mode = input("\nPlease enter the correct characters : ")

      if mode == "1":
        register = Register()
        register.main()
      elif mode == "2":
        setup = setup_mode()
        setup.setup_mode_select()

      elif mode == "3":
        pass

# 以下池田作成

class Random_model():
    def __init__(self):
        self.loaded_model = 100
    def image_transform(self, pil_image=None):
        #　モデルに応じてリサイズ処理をしてインスタンス変数
        self.image_array = np.random.randn(100, 100)
    def predict(self):
        # インスタンス変数のarrayから予測、予測されたラベルを返す
        random_index = np.random.choice(10, np.random.randint(1, 4))
        return list(random_index)

class Register(main):
    def __init__(self):
        # 親クラスのインスタンス変数を読み込み
        super().__init__()
        # bottle_master_dfを呼び込む
        self.bottle_master_df = bottle_master.bottle_master_df()
        
      

    def main(self):
        #　モデルのオブジェクトを呼び込む、モデルクラス後に追加
        print(self.model)
        if self.model == "Random":
            self.model_object = Random_model()
        else:
            pass
        display(self.bottle_master_df)
        while True:
            # 空のdfを作成
            total_df = pd.DataFrame(columns=['name', 'price'])
            while True:
                # 「商品を置いてenterを押してください」
                scene = "scan_start"
                input(translate_language(self.lang, scene))

                # 時間を図る
                # self.time_debugに応じて対応
                if self.time_debug:
                    time_start = perf_counter()
                # 写真撮影をしてnumpy_arrayで受け取る
# image_array = shutter2image.shutter_array()

                # numpy_arrayをPILへ変換してモデルの入力用に加工
# img = Image.formarray(image_array)
                img = None
                # 画像表示
# img.show()
                
                # modelオブジェクトのimage_transformでPILをresize, numpy_arrayに変換
                self.model_object.image_transform(pil_image=img)
                
                # 予測されたindexのリストを読み込む
                predict_index_list = self.model_object.predict()
                print('debug 予測されたラベルインデックスのリスト {}'.format(predict_index_list))
                # 予測されたindexのリストからdfを作成
                predict_df = self.bottle_master_df.iloc[predict_index_list, [2, 3]].reset_index(drop=True)
                
                # 小計をdiplay
                scene = "read_message"
                # print(translate_language(self.lang, scene).format('test_bottle', random_price))
                print('読み取られた商品の一覧は以下になります')
                display(predict_df)

                # 時間計測
                if self.time_debug:
                    time_end = perf_counter()
                    # 商品を読み取るまで {}[s] 経過しました
                    scene = "time"
                    print(translate_language(self.lang, scene).format(time_end-time_start))

                while True:
                # 読み取られた商品が正しい場合は「y」、誤っていた場合は「n」を押してください
                    scene = "correct?"
                    key = input(translate_language(self.lang, scene))

                    if key != 'y' and key != 'n':
                        # 正しく入力されていません
                        scene = "input_error"
                        print(translate_language(self.lang, scene))
                    else:
                        break
                
                if key == 'y':
                    total_df = pd.concat([total_df, predict_df], axis=0).reset_index(drop=True)
                    display(total_df)
                    # 小計
                    scene = "subtotal"
                    print(translate_language(self.lang, scene).format(total_df["price"].sum()))

                    while True:
                        # 続けて商品をスキャンする場合は「y」、会計する場合は「f」、\n キャンセルする商品がある場合は「ｘ」を押して下さい。
                        scene = "continue?"
                        key = input(translate_language(self.lang, scene))
                        if key != 'y' and key != 'f' and key != 'x':
                            # 正しく入力されていません
                            scene = "input_error"
                            print(translate_language(self.lang, scene))
                        # 商品をdeletする処理
                        elif key == 'x':
                            while True:
                                display(total_df)
                                # scene設定
                                key = input('商品から消去する商品の番号を押してください, 消去しない場合は「n」を押してください')
                                key_list = [str(i) for i in range(len(total_df))] + ['n']
                                if key not in key_list:
                                    # 正しく入力されていません
                                    scene = "input_error"
                                    print(translate_language(self.lang, scene))
                                elif key == 'n':
                                    break
                                else:
                                    print('以下の商品を消去しました')
                                    display(total_df[int(key):int(key)+1])
                                    #　total_dfから消去
                                    total_df = total_df.drop(index=int(key))
                                    #　indexをreset
                                    total_df.reset_index(drop=True)
                        else:
                            break

                    if key == 'f':
                        # 合計
                        display(total_df)
                        scene = "total"
                        print(translate_language(self.lang, scene).format(total_df['price'].sum()))

                        # ありがとうございました
                        scene = "thanks"
                        print(translate_language(self.lang, scene))
                        print()
                        break
                    elif key == 'y':
                        # 次の商品を指定の位置に置いてください。
                        scene = "next"
                        print(translate_language(self.lang, scene))

if __name__ == '__main__':
    main().mode_select()


----------Current status---------
model : Random
language : Japanese

----------Please choice start up mode----------
 1 : Start in cash register mode
 2 : Start in set up mode
 3 : Start in image mode
--------------------
Please select and enter the number : １

Invalid number.

Please enter the correct characters : 

Invalid number.

Please enter the correct characters : 1
Random


,id,revisiondate,name,price
0,0,2020-10-01 00:00:00.000000,cola,120
1,1,1900-01-01 00:00:00.000000,namacha,200
2,2,1900-01-01 00:00:00.000000,calpis,150
3,3,1900-01-01 00:00:00.000000,gogotea,130
4,4,1900-01-01 00:00:00.000000,pocari,140
5,5,1900-01-01 00:00:00.000000,cola_l,150
6,6,1900-01-01 00:00:00.000000,namacha_l,250
7,7,1900-01-01 00:00:00.000000,calpis_l,200
8,8,1900-01-01 00:00:00.000000,gogotea_l,180
9,9,1900-01-01 00:00:00.000000,pocari_l,190


スキャンする場合は商品を指定の位置に置いて「Enter」を押して下さい。
debug 予測されたラベルインデックスのリスト [0]
読み取られた商品の一覧は以下になります


,name,price
0,cola,120


商品を読み取るまで 0.00787348900030338[s] 経過しました。
読み取られた商品が正しい場合は「y」、誤っていた場合は「n」を押してください。 : y


...

,name,price
0,cola,120


,name,price


商品から消去する商品の番号を押してください, 消去しない場合は「n」を押してくださいn
続けて商品をスキャンする場合は「y」、会計する場合は「f」、
 キャンセルする商品がある場合は「ｘ」を押して下さい。 : x


,name,price


KeyboardInterrupt: ignored

In [26]:
a = Register()
a.model_dict

{'e': 'EFF', 'y': 'YOLO'}

In [68]:
from bottle_master import bottle_master
bottle_master.bottle_master_df()

,id,revisiondate,name,price
0,0,2020-10-01 00:00:00.000000,cola,120
1,1,1900-01-01 00:00:00.000000,namacha,200
2,2,1900-01-01 00:00:00.000000,calpis,150
3,3,1900-01-01 00:00:00.000000,gogotea,130
4,4,1900-01-01 00:00:00.000000,pocari,140
5,5,1900-01-01 00:00:00.000000,cola_l,150
6,6,1900-01-01 00:00:00.000000,namacha_l,250
7,7,1900-01-01 00:00:00.000000,calpis_l,200
8,8,1900-01-01 00:00:00.000000,gogotea_l,180
9,9,1900-01-01 00:00:00.000000,pocari_l,190


In [67]:
random_index = np.random.choice(10, np.random.randint(1, 4))
list(random_index)

[3, 6, 2]

In [69]:
df = bottle_master.bottle_master_df()
df.iloc[list(random_index), [2, 3]]

,name,price
3,gogotea,130
6,namacha_l,250
2,calpis,150


In [86]:
!git pull

remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 54 (delta 2), reused 54 (delta 2), pack-reused 0
Unpacking objects: 100% (54/54), done.
From https://github.com/nakazono1011/dive_groupwork
   da2ed36..5f56942  master     -> origin/master
Updating da2ed36..5f56942
^C
